In [1]:
pip install pandas numpy scikit-learn tensorflow

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
final_df = pd.read_pickle('/content/final_data (1).pkl')

In [4]:
final_df.head()

,user_id,product_id,reordered,user_product_reorder_count,reorder_ratio,times_reordered,avg_days_between_orders,total_orders
0,1,196,1,9,0.77648,27791,18.542373,10
1,1,25133,1,7,0.740155,4586,18.542373,10
2,1,38928,1,0,0.827769,10958,18.542373,10
3,1,26405,1,1,0.441516,536,18.542373,10
4,1,39657,1,0,0.766288,3846,18.542373,10


In [5]:
final_df['reordered'].unique()

<IntegerArray>
[1, 0]
Length: 2, dtype: Int64

In [6]:
y = final_df['reordered']
x = final_df.drop(columns=['reordered','user_id', 'product_id'])

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, stratify=y, test_size= 0.25)

In [8]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(1038462, 5) (1038462,)
(346155, 5) (346155,)


In [9]:
y_train.value_counts()

,count
reordered,
1,621618
0,416844


In [10]:
print(y_test.value_counts())

reordered
1    207206
0    138949
Name: count, dtype: Int64


In [11]:
np.ceil(y_train.value_counts()[0]/y_train.value_counts()[1])

np.float64(1.0)

In [12]:
sc = StandardScaler()
x_train_sc = sc.fit_transform(x_train)
x_test_sc = sc.transform(x_test)

In [13]:
model = Sequential()

model.add(Dense(128, activation = 'relu', input_shape=(x_train_sc.shape[1],)))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation ='sigmoid'))

model.compile(optimizer = "adam", loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,649 (45.50 KB)

 Trainable params: 11,649 (45.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
x_train_sc = x_train_sc.astype('float32')
x_test_sc = x_test_sc.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [15]:
result = model.fit(x_train_sc, y_train, validation_data=(x_test_sc, y_test), epochs=50, batch_size=512)

Epoch 1/50
2029/2029 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.8121 - loss: 0.3731 - val_accuracy: 0.8264 - val_loss: 0.3441
Epoch 2/50
2029/2029 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8276 - loss: 0.3428 - val_accuracy: 0.8263 - val_loss: 0.3443
Epoch 3/50
2029/2029 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.8279 - loss: 0.3424 - val_accuracy: 0.8264 - val_loss: 0.3438
Epoch 4/50
2029/2029 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.8272 - loss: 0.3428 - val_accuracy: 0.8264 - val_loss: 0.3438
Epoch 5/50
2029/2029 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.8274 - loss: 0.3429 - val_accuracy: 0.8264 - val_loss: 0.3443
Epoch 6/50
2029/2029 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8280 - loss: 0.3422 - val_accuracy: 0.8264 - val_loss: 0.3439
Epoch 7/50
2029/2029 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.8277 - loss: 0.3424 - val_accuracy: 0.8264 - val_loss: 0.3438
Epoch 8/50
2029/2029 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.8273 - loss: 0

In [16]:
print("x_train size:", len(x_train))

x_train size: 1038462


In [18]:
y_pred = model.predict(sc.transform(x_test).astype('float32')).flatten()
y_pred_class = (y_pred > 0.5).astype(int)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred_class))
print("\nROC AUC Score:", roc_auc_score(y_test, y_pred))

10818/10818 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step

Classification Report:

              precision    recall  f1-score   support

         0.0       0.70      1.00      0.82    138949
         1.0       1.00      0.71      0.83    207206

    accuracy                           0.83    346155
   macro avg       0.85      0.85      0.83    346155
weighted avg       0.88      0.83      0.83    346155


ROC AUC Score: 0.8844466303916417


In [19]:
model.save("reorder_prediction_model.h5")